## Collect a Company's Patent Porfolio

This notebook will attempt to amass, for any given company, their entire worldwide patent porfolio.\*

### U.S. Portfolio

Collecting their U.S. portfolio will proceed in a few steps:

1. Collect all applications ever assigned to the company

    a. Those applications naming the company as an applicant
    b. Those assigned to the company via an assignment
    
2. Prune all applications assigned out of the company, as recorded in USPTO assignment records

\*NOTE: I say "attempt" because the notebook relies on the USPTO system, and the International Patent Document (INPADOC) service. Not all foreign jurisdictions participate in INPADOC, and not all INPADOC member states are as good at contributing to the system as we would like. So coverage is good, but not perfect.

First, we do our standard imports, and grab the USApplication and Assignment objects from patent_client

In [1]:
import pandas as pd
from patent_client import USApplication, Assignment

company_name = 'Tesla Motors'

### Step 1.a: Collect all applications naming the company as applicant

In [3]:
applicant_apps = USApplication.objects.filter(first_named_applicant=company_name).values_list('appl_id', flat=True).to_list()

### Step 1.b: Collect all applications assigned to the company

In [5]:
# The assigned apps is either a single value, or a list of values if more than one property was assigned
assigned_apps = Assignment.objects.filter(assignee=company_name).explode('properties').values_list('appl_id', flat=True).to_list()

And now we combine the two

In [6]:
all_apps = set(list(applicant_apps + assigned_apps))

print(f'Total U.S. Applications Ever Owned by {company_name}: {len(all_apps)}')

Total U.S. Applications Ever Owned by Tesla Motors: 242


### Step 2: Collect all applications ever assigned out of the company

In [7]:
assigned_out_apps = set(Assignment.objects.filter(assignor=company_name).to_pandas()
                   .query('conveyance_text == "ASSIGNMENT OF ASSIGNORS INTEREST"')
                   .properties.explode().apply(lambda x: x.appl_id).to_list())
print(f'U.S. Applications assigned out of {company_name}: {len(assigned_out_apps)}')

U.S. Applications assigned out of Tesla Motors: 14


### Step 3: Subtract one set from the other

In [11]:
owned_apps = list(set(all_apps) - set(assigned_out_apps))
len(owned_apps)


232

### Step 4: Generate status report

In [12]:
import pandas as pd
from patent_client.parser import parse

us_df = USApplication.objects.filter(appl_id=owned_apps).values(
    'appl_id', 
    'app_filing_date', 
    'app_early_pub_number',
    'app_early_pub_date',
    'patent_number',
    'patent_issue_date',
    'patent_title',
    'inventors__0__name',
    'applicants__0__name',
    'app_status',
    'app_status_date'
).to_pandas()

us_df.head()

,appl_id,app_filing_date,app_early_pub_number,app_early_pub_date,patent_number,patent_issue_date,patent_title,inventors__0__name,applicants__0__name,app_status,app_status_date
0,15334090,2016-10-25,US20180114740A1,2018-04-26,None,None,INVERTER,None,None,Notice of Allowance Mailed -- Application Rece...,2022-06-09
1,14070214,2013-11-01,US20150123511A1,2015-05-07,10938280,2021-03-02,Flux Shield for Electric Motor,None,None,Patented Case,2021-02-10
2,15381259,2016-12-16,US20170096073A1,2017-04-06,None,None,CHARGING STATION PROVIDING THERMAL CONDITIONIN...,None,None,Abandoned -- After Examiner's Answer or Board ...,2021-01-21
3,14706837,2015-05-18,US20150244047A1,2015-08-27,None,None,Battery mounting and cooling system,None,None,Abandoned -- Failure to Respond to an Office A...,2020-06-19
4,15382451,2016-12-16,US20170184345A1,2017-06-29,10641552,2020-05-05,HEAT-RECOVERING TEMPERATURE-GRADIENT BASED OVE...,None,None,Patented Case,2020-04-15


### Foreign Patent Portfolio

Now we will fetch the company's wordwide portfolio, as it appears in the Inpadoc system maintained by the EPO

In [15]:
from patent_client import Inpadoc

foreign = Inpadoc.objects.filter(applicant=company_name)

And now, a status report:

In [16]:
foreign_df = foreign[:15].values(
    'country', 
    appl_id='biblio.applications.0.number', 
    app_filing_date='biblio.applications.0.date', 
    pub_number='biblio.publications.0.number', 
    pub_date='biblio.publications.0.date', 
    title='biblio.title', 
    applicant='biblio.applicants.0', 
    first_named_inventor='biblio.inventors.0').to_pandas()
foreign_df = foreign_df[foreign_df['country'] != 'US']
foreign_df

,country,appl_id,app_filing_date,pub_number,pub_date,title,applicant,first_named_inventor
1,KR,None,None,None,None,None,None,None
2,KR,None,None,None,None,None,None,None
6,CN,None,None,None,None,NOVEL BATTERY SYSTEMS BASED ON LITHIUM DIFLUOR...,None,None
7,KR,None,None,None,None,None,None,None
9,CA,None,None,None,None,NOVEL BATTERY SYSTEMS BASED ON LITHIUM DIFLUOR...,None,None
10,WO,None,None,None,None,ELECTROLYTES WITH LITHIUM DIFLUORO(OXALATO)BOR...,None,None
11,CA,None,None,None,None,DIOXAZOLONES AND NITRILE SULFITES AS ELECTROLY...,None,None
12,WO,None,None,None,None,DIOXAZOLONES AND NITRILE SULFITES AS ELECTROLY...,None,None
14,WO,None,None,None,None,METHOD FOR SYNTHESIZING NICKEL-COBALT-ALUMINUM...,None,None


And now we will take our results, and write to an excel file

In [17]:
writer = pd.ExcelWriter(f'{company_name}_portfolio.xlsx')
us_df.to_excel(writer, sheet_name='US Portfolio')
foreign_df.to_excel(writer, sheet_name='Foreign Portfolio')
writer.save()